# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [89]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        self.n_classes = len(np.unique(y))
        self.y = y
        
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        изучите работу sklearn.neighbors.NearestNeighbors'''
        
        distances, neigh_ind = self.NearestNeighbors.kneighbors(X)
        
        # получим здесь расстояния до соседей distances и их метки
        
        classes_ = [self.n_classes]
        _y = self.y

        n_samples = X.shape[0]
       
        
        if self.weights == 'uniform':
            w = np.ones(distances.shape)
        else:
            # чтобы не делить на 0, 
            # добавим небольшую константу, например 1e-3
            w = 1/(distances + 1e-3)

        all_rows = np.arange(X.shape[0])    
            
        probs = []
        for k, classes_k in enumerate(classes_):
            pred_labels = _y[:, k][neigh_ind]
            proba_k = np.zeros((n_samples, classes_k.size))

            # a simple ':' index doesn't work right
            for i, idx in enumerate(pred_labels.T):  # loop is O(n_neighbors)
                proba_k[all_rows, idx] += weights[:, i]

            # normalize 'votes' into real [0,1] probabilities
            normalizer = proba_k.sum(axis=1)[:, np.newaxis]
            normalizer[normalizer == 0.0] = 1.0
            proba_k /= normalizer

            probs.append(proba_k)

        if not self.outputs_2d_:
            probs = probs[0]       
            
        # реализуем вычисление предсказаний:
        # выбрав один объект, для каждого класса посчитаем
        # суммарный вес голосующих за него объектов
        # затем нормируем эти веса на их сумму
        # и вернем это как предсказание KNN
        return probs

# Загрузим данные и обучим классификатор

In [92]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, )


IndexError: too many indices for array

Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [5]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

NameError: name 'prediction' is not defined

Несколько примеров, на которых можно проверить правильность реализованного метода:

In [6]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

NameError: name 'inds' is not defined

# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [7]:
for i in 56, 83, 127:
    print('{:.4f}'.format(max(prediction[i])))

NameError: name 'prediction' is not defined